<a href="https://colab.research.google.com/github/ras1364/Fake-Review-Using-BERT/blob/main/FakeReviewDetectionUsingBertVer3_3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**In this code, comments are sent to the BERT encoder regardless of the number of sentences, and its output is sent to the dence network.**

In [20]:
import pandas as pd
import numpy as np

In [4]:
from google.colab import files
uploaded = files.upload()

Saving X_test.csv to X_test.csv
Saving X_train.csv to X_train.csv
Saving y_test.csv to y_test.csv
Saving y_train.csv to y_train.csv


In [5]:
X_test = pd.read_csv("/content/X_test.csv")
X_train = pd.read_csv("/content/X_train.csv")
y_test = pd.read_csv("/content/y_test.csv")
y_train = pd.read_csv("/content/y_train.csv")

In [14]:
# Number of sentece in each review 
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from gensim.parsing.preprocessing import remove_stopwords


def SpliteAndPreprocess(text):
  review=[]
  splited_review = sent_tokenize(text)
  for str in splited_review:
    review.append(remove_stopwords(str))
  return review


X_train["stemmed_text_data"] = X_train["text"].map(SpliteAndPreprocess)
X_test["stemmed_text_data"] = X_test["text"].map(SpliteAndPreprocess)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
X_train["stemmed_text_data"][0]

['I recently stayed Hard Rock Hotel Chicago town business, enjoyed stay much.',
 'I stayed room king bed, comfortable.',
 'The room things I needed stay, nice.',
 'The thing I think party floor, quieted quickly enough.',
 'My room internet connection!',
 'I dinner night China Grill, amazing meal!',
 'Overall, I great stay, return time I Chicago.']

In [35]:
max_length = 0
max_sentence_length = 0
for x in X_train["stemmed_text_data"]:
  if len(x) > max_length:
    max_length = len(x)
  for sen in x:
    if len(sen) > max_sentence_length:
      max_sentence_length = len(sen)
print("Max length of sentence = ",max_sentence_length , " Max number of sentence in review = ", max_length)
max_sentence_length = 150

Max length of sentence =  861  Max number of sentence in review =  49


In [18]:
from transformers import AutoTokenizer, TFBertModel 
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [51]:
def encode(text, maxlen):
  input_ids=[]
  attention_masks=[]

  for row in text:
    ids = []
    mask = []
    for sent in row:
      encoded = tokenizer.encode_plus(
          sent,
          add_special_tokens=True,
          max_length=maxlen,
          pad_to_max_length=True,
          return_attention_mask=True,
      )
      ids.append(encoded['input_ids'])
      mask.append(encoded['attention_mask'])
    loop = 50 - len(ids)
    for x in range(loop):
      ids.append([0]*max_sentence_length) 
      mask.append([0]*max_sentence_length) 

    input_ids.append(ids)
    attention_masks.append(mask)

  return np.array(input_ids),np.array(attention_masks)

In [52]:
X_train_input_ids, X_train_attention_masks = encode(X_train.stemmed_text_data.tolist(), maxlen=max_sentence_length)
X_test_input_ids, X_test_attention_masks = encode(X_test.stemmed_text_data.tolist(), maxlen=max_sentence_length)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [53]:
print("X_train.shape = " , X_train_input_ids.shape, " y_train.shape = ", y_train.deceptive.shape)
print("X_test.shape = " , X_test_input_ids.shape, " y_test.shape = ", y_test.deceptive.shape)

X_train.shape =  (1200, 50, 150)  y_train.shape =  (1200,)
X_test.shape =  (400, 50, 150)  y_test.shape =  (400,)


In [54]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

def build_model(bert_model):
   input_word_ids = tf.keras.Input(shape=(max_length,max_sentence_length,),dtype='int32')
   attention_masks = tf.keras.Input(shape=(max_length,max_sentence_length,),dtype='int32')

   sequence_output = bert_model([input_word_ids,attention_masks])
   output = sequence_output[1]
   output = tf.keras.layers.Dense(32,activation='relu')(output)
   output = tf.keras.layers.Dropout(0.2)(output)
   output = tf.keras.layers.Dense(1,activation='sigmoid')(output)

   model = tf.keras.models.Model(inputs = [input_word_ids,attention_masks], outputs = output)
   model.compile(Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

   return model

In [55]:
model = build_model(bert_model)
model.summary()

ValueError: ignored

In [ ]:
history = model.fit(
    [X_train_input_ids, X_train_attention_masks],
    y_train.deceptive,
    batch_size=32,
    epochs=5,
    validation_data=([X_test_input_ids, X_test_attention_masks], y_test.deceptive))

In [ ]:
loss, accuracy = model.evaluate([X_test_input_ids, X_test_attention_masks], y_test.deceptive)
print('Test accuracy :', accuracy ,' and loss = ', loss)

In [ ]:
y_predicted = model.predict([X_test_input_ids, X_test_attention_masks])
y_predicted = y_predicted.flatten()

In [ ]:
import numpy as np
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test.deceptive, y_predicted)
cm 

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test.deceptive, y_predicted))